# Linear Regression

In this homework we are going to apply linear regression to the problem of predicting developer satisfaction based upon information about their carrers, from a StackOverflow survey.  The data from this question is based on the [2019 StackOverflow Survey](https://insights.stackoverflow.com/survey/2019); accordingly, the subset bundled with this assignment is also released under the Open Database License (ODbL) v1.0.  For this problem, you should not use Scikit-Learn, but instead implement all the least squares solutions manually.


### Q1 Data Parsing

The data from this survey has the values as described below.  Your eventual goal will be to predict (a numerical equivalent of) the `CarreerSat` column, indicating how satisfied the subject is with their carreer), based upon the other values.  Note that because the career satisfaction values are ordinal, we likely should not be predicting them with linear regression, but as an illustrative example, this is still a reasonable task.

The data set contains the following columns.

| Column | Sample | Does/is the respondent... | Type/Values |
| --- |:--- |:--- |:--- |
| **CareerSat** | 'vs' | satisfied with their career? | (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) -- corresponding to ({very, slightly}, {satisfied, dissatisfied}), neutral, and not applicable |
| MgrWant | 'n' | ...want to be a manager? | boolean |
| Age    | '22'   | age | integer     |
| CodeRevHrs | '2' | hours a week spent reviewing code | integer |
| ConvertedComp | '61000' | yearly compensation in 2019 USD | integer |
| Country | 'United States' | lives in country | string _(ignore in regression)_ |
| Dependents | 'n' | ...have children or other dependents. | boolean |
| DevEnvironVSC | 'y' | ...use Visual Studio Code | boolean |
| DevTypeFullStack | 'n' | ...identify as a full-stack developer | boolean |
| EdLevel | 'bachelors' | maximum education level | (`other`, `bachelors`, `masters`, `doctoral`) |
| EduOtherMOOC | 'y' | ...ever taken a Massively Open Online Course | boolean |
| EduOtherSelf | 'y' | ...ever taught themselves a new platform | boolean _(ignore in regression, this is  duplicate field)_ |
| Extraversion | 'y' | ...prefer in-person meetings to online meetings | boolean |
| GenderIsMan | 'y' | ...male | boolean |
| Hobbyist | 'n' | ...write code as a hobby? | boolean |
| MgrIdiot | 'very' | ...think their manager knows what they are doing? | (`NA`, `not`, `some`, `very`), in order of increasing confidence |
| OpSys | 'win' | which OS do they use? | (`win`, `mac`, `tux`, `NA`), for (Windows, Mac OSX, Linux-like, NA) |
| OpenSourcer | 'Never' | ...contribute to open-source projects? | (`never`, `year`, `month-year`, `month`), in increasing order of frequency |
| OrgSize | '100-499' | number of employees in organization? | (`NA`, `1`, `2-9`, `10-19`, `20-99`, `100-499`, `500-999`, `1,000-4,999`, `5,000-9,999`, `10,000+`) |
| Respondent | '4' | respondent ID from original data | integer _(ignore in regression)_ |
| Student | 'n' | ...currently a student? | boolean |
| UndergradMajorIsComputerScience | 'y' | ...majored in CS? | boolean |
| UnitTestsProcess | 'n' | ...use unit tests in their job? | boolean |
| WorkWeekHrs | '40' | hours a week worked | integer |
| YearsCode | 3 | years since first programming | integer |
| YearsCodePro | 0 | years programming professionally | integer |

When you load the data for linear regression, you will want to convert each type to a floating point value according to the colummns above.  Specifically, for each column listed above, convert the value to a numeric quantity using the rules below.  Note that there are some fairly un-natural assumptions here (like converting NAs to 0.0), but these are largely to illustrate the methodology without adding too much additional processing work.

 - _boolean_ : `y`/`NA`/`n` assigned to `+1.0`/`0.0`/`0.0`
 - _integer_ : convert to `float`, preserving value. `NA` equals `0.0`. 
 - _string_ : not included in regression; we'll use it later
 - CareerSat: Map (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) to (-2.0, -1.0, 0.0, 0.0, 1.0, 2.0)
 - EdLevel: Map (`other`, `bachelors`, `masters`, `doctoral`) to (0.0, 1.0, 1.5, 2.0)
 - MgrIdiot: Map (`NA`, `not`, `some`, `very`) to (-1.0, -1.0, 0.0, 1.0)
 - OpSys: Map (`win`, `mac`, `NA`, `tux`, `BSD`) to (-1.0, 0.0, 0.0, 1.0, 1.0)
 - OpenSourcer : Map (`never`, `year`, `month-year`, `month`) to (0.0, 0.5, 1.0, 2.0)
 - OrgSize: Map each range "$a$-$b$" to the value $ln(a)$. Treat `NA` as `ln(1.0) = 0`. We are converting an exponentially distributed range to a linearly distributed one.

Remove the columns listed above as being ignored. 

Some hints:
  1. Load the csv file with `pd.read_csv(filname, dtype=str, keep_default_na=False)` to ensure that you load all columns as text (so you can do your own preprocessing), and ignore pandas's default conversion to NaN values.
  2. Use the `.apply()` function in pandas to convert

In [2]:
import csv
import gzip
import math
import hashlib
import numpy as np
import pandas as pd
import mugrade

In [44]:
#@mugrade.test_local
@mugrade.submit("QlIuTCr8C4BPu7aXrk73")
def parse_stackoverflow_data():
    """Load data from the "eggs.csv.gz" file, and convert for use in linear regression
    
    Returns:
        pandas.DataFrame, containing the data converted to floating point values appropriately.
    """
    df = pd.read_csv('eggs.csv.gz', dtype=str, keep_default_na=False, compression = 'gzip')
    df = df.drop(columns = ['Country', 'EduOtherSelf', 'Respondent'])
    df['CareerSat'] = df['CareerSat'].map({'vd': -2.0, 'sd': -1.0, 'ne': 0.0, 'NA': 0.0, 'ss': 1.0, 'vs':2.0})
    df['EdLevel'] = df['EdLevel'].map({'other':0.0, 'bachelors':1.0, 'masters':1.5, 'doctoral':2.0})
    df['MgrIdiot'] = df['MgrIdiot'].map({'NA':-1.0, 'not':-1.0, 'some':0.0, 'very':1.0})
    df['OpSys'] = df['OpSys'].map({'win':-1.0, 'mac':0.0, 'NA':0.0, 'tux':1.0, 'BSD':1.0})
    df['OpenSourcer'] = df['OpenSourcer'].map({'never':0.0, 'year':0.5, 'month-year':1.0, 'month':2.0})
    df['OrgSize'] = df['OrgSize'].map(lambda x: 0.0 if x == 'NA' else math.log(float(x.replace("+", "").replace(",", "").split("-")[0])))
    df = df.replace('y', 1.0)
    df = df.replace('n', 0.0)
    df = df.applymap(lambda x : 0.0 if x == 'NA' else x)
    df = df.apply(lambda x : x.astype(np.float64))
    return df

### Submitting 3 grader tests
# Running test 1/3 ... PASSED
# Running test 2/3 ... PASSED
# Running test 3/3 ... PASSED


### Q2 Splitting Data

Now we prepare the converted data for regression. In this step, we:

 1. Extract the data as a numpy array
 2. Split the data into train and validation sets.  You can use the first 20% of the data (rounded down) as the validation set and keep the remaining as the training set. (Note that it is common practice to randomize the dataset; this has already been done. Don't shuffle the dataset for this assignment.)
 3. Split each set into the predicted column (the first column in the data frame) and the feature columns (the remaining columns), plus a final column corresponding to a constant 1.0 value.  Not that you should keep the ordering of the feature columns the same as they appear in the data.


In [60]:
#@mugrade.test_local
@mugrade.submit("QlIuTCr8C4BPu7aXrk73")
def split_data(df):
    """split the data into training and validation sets, and convert them to np.ndarray. (Step 1 and 2 above.)

    args:
        df : pandas.DataFrame -- the parsed data, as returned by parse_stackoverflow_data()

    returns: X_train, y_train, X_val, y_val
      X_train  : np.ndarray -- the second 80% of the data features
      y_train : np.ndarray -- the second 80% of the target values
      X_val : np.ndarray -- the first 20% (rounded down) of the data features
      y_val : np.ndarray -- the first 20% of the target values
    """
    df = df.to_numpy()
    df = np.concatenate((df, np.full((len(df), 1), 1.0)), axis = 1)
    test = df[0:int(len(df)*(20/100)), :]
    train = df[int(len(df)*(20/100)):len(df), :]
    X_train = train[:, 1:]
    y_train = train[:, 0]
    X_val = test[:, 1:]
    y_val = test[:, 0]
    return X_train, y_train, X_val, y_val

### Submitting 3 grader tests
# Running test 1/3 ... PASSED
# Running test 2/3 ... PASSED
# Running test 3/3 ... PASSED


### Q3 Linear Regression

Now we are going to build a simple scikit-learn-like class for least squares linear regression.  Recall from lecture that the linear regression approach models the data as
$$ y^{(i)} \approx \theta^T x^{(i)} $$
and the optimal $\theta$ is given by
$$ \theta^\star = (X^T X)^{-1}X^T y $$
using the notation described in the slides and course notes.  Recall, as mentioned in class, that you should use the `np.linalg.solve()` function rather than the `np.linalg.inv()` function to compute this solution.

Implement the class below, plus the `squared_error` function.

In [74]:
def squared_error(y_pred, y):
    """ Utility function to compute squared error
    args:
        y_pred : np.ndarray[num_examples] -- the predictions
        y : np.ndarray[num_examples] -- the ground truth values

    returns:
        float : _average_ squared error between y_pred and y
    """
    return np.mean((y_pred - y)**2)
    
#@mugrade.test_local
@mugrade.submit("QlIuTCr8C4BPu7aXrk73")
class LinearRegression():
    """ Perform linear regression and predict the output on unseen examples. 
    
    attributes: 
        theta (np.ndarray) : vector containing parameters for the features
    """

    def __init__(self, X, y):
        """ Train the linear regression model by computing the estimate of the parameters
        You should store the model parameters in self.theta

        args: 
            X (np.ndarray[num_examples, num_columns]) : matrix of training data
            y (np.ndarray[num_examples]) : vector of output variables

        return: LinearRegression -- returns itself (for convenience)
        """
        self.theta = np.linalg.solve(X.T @ X, X.T @ y)
        

    def predict(self, X): 
        """ Use the learned model to predict the output of X_p

        args: 
            X : np.ndarray[num_examples, num_columns] -- matrix of features for which we form a prediction

        return: 
            np.ndarray[num_examples], vector of predicted outputs
        """
        return X @ self.theta
    
    
    

### Submitting 2 grader tests
# Running test 1/2 ... PASSED
# Running test 2/2 ... PASSED


## Q4 Evaluation versus baselines

As a final consideration, If you implement this properly, you will see that we get a squared error of around 1.3 on the validation set.  Is this "good"?  This is one of the more subtle points of many data science problems, but we can start to get some sense of this by looking at what the predictions would look like if we _just_ predicted the mean target output on the training set (this is essentially the "simplest" prediction we could make, if we didn't look at the features at all).

Implement the following function to evaluate our linear regression.  Think about what this signifies about the quality of the solution.

In [80]:
#@mugrade.test_local
@mugrade.submit("QlIuTCr8C4BPu7aXrk73")
def evaluate_linear_regression(X_train, y_train, X_val, y_val):
    """ Evaluate the squared error of linear regression versus the simple mean-prediciton baseline.
    
    Args: X_train, y_train, X_val, y_val -- output of split_data() function
    
    Return: Tuple[validation_mse, baseline_mse]:
        validation_mse: float -- squared error of predictions on validation set, when training on training set
        baseline_mse: float -- squared error of predicting the mean on the training set
    """
    pred = LinearRegression(X_train, y_train).predict(X_val)
    validation_mse = squared_error(pred, y_val)
    baseline_mse = squared_error(np.mean(y_train), y_val)
    return (validation_mse, baseline_mse)

### Submitting 1 grader tests
# Running test 1/1 ... PASSED
